<a href="https://colab.research.google.com/github/rajeevo/Load_-_Preprocess_Data_CSV/blob/master/Load_%26__Preprocess_Data_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 1.) Import Libraries

In [6]:
import functools

import numpy as np
import tensorflow as tf
import pandas as pd

In [7]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [8]:
# Make numpy values easier to read
np.set_printoptions(precision=3, suppress=True)

In [17]:
# 2.) Load Data

In [10]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [12]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [14]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key, value.numpy()))

In [15]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'female' b'male']
age                 : [36. 28. 28. 28. 39.]
n_siblings_spouses  : [0 1 0 1 0]
parch               : [0 0 0 2 0]
fare                : [ 7.896 15.5    0.    23.45   0.   ]
class               : [b'Third' b'Third' b'Second' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'A']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'n' b'y']


In [16]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [50.  34.  50.  32.5 24. ]
n_siblings_spouses  : [1 1 0 1 0]
parch               : [0 0 1 0 1]
fare                : [ 55.9    21.     26.     30.071 247.521]
class               : [b'First' b'Second' b'Second' b'Second' b'First']
deck                : [b'E' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Cherbourg']
alone               : [b'n' b'n' b'n' b'n' b'n']


In [17]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [40.    0.83 51.   47.   16.  ]
n_siblings_spouses  : [0 0 0 0 0]
class               : [b'Third' b'Second' b'First' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'n' b'n' b'y' b'y']


In [ ]:
# 3.) Data Preprocessing

In [18]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28. 28. 49. 70. 33.]
n_siblings_spouses  : [1. 1. 1. 1. 1.]
parch               : [0. 0. 0. 1. 2.]
fare                : [ 19.967 133.65   76.729  71.     27.75 ]


In [19]:
example_batch, labels_batch = next(iter(temp_dataset))

In [20]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [21]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 26.      0.      0.      8.05 ]
 [ 28.      0.      0.    110.883]
 [ 28.      0.      0.      7.896]
 [ 28.      0.      0.     26.55 ]
 [ 45.      1.      1.    164.867]]

[0 1 0 0 1]


In [22]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [51. 34. 36. 70. 35.]
n_siblings_spouses  : [0 0 0 1 0]
parch               : [0 0 0 1 0]
fare                : [ 7.054 26.55   0.    71.    26.288]
class               : [b'Third' b'First' b'Third' b'First' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'B' b'E']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']


In [23]:
example_batch, labels_batch = next(iter(temp_dataset))

In [24]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [25]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [26]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
class               : [b'Third' b'First' b'Second' b'Third' b'Third']
deck                : [b'unknown' b'D' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'n' b'n' b'y' b'y']
numeric             : [[20.     0.     0.     7.05 ]
 [47.     1.     1.    52.554]
 [23.     2.     1.    11.5  ]
 [19.     0.     0.     7.896]
 [15.     0.     0.     7.225]]


In [27]:
example_batch, labels_batch = next(iter(packed_train_data))

In [ ]:
# 4.) Data Normalization

In [28]:
import pandas as pd

In [29]:
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [31]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [32]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [35]:
# See what you just created
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fe5ff624158>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [36]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[32.   ,  1.   ,  1.   , 15.5  ],
       [26.   ,  0.   ,  0.   ,  7.896],
       [28.   ,  1.   ,  0.   , 24.   ],
       [23.   ,  0.   ,  0.   , 13.   ],
       [28.   ,  0.   ,  0.   , 13.   ]], dtype=float32)>

In [37]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.189,  0.395,  0.782, -0.346],
       [-0.29 , -0.474, -0.479, -0.485],
       [-0.13 ,  0.395, -0.479, -0.19 ],
       [-0.53 , -0.474, -0.479, -0.392],
       [-0.13 , -0.474, -0.479, -0.392]], dtype=float32)

In [38]:
# 5.) Categorical Data

In [40]:
CATEGORIES = {
    'sex' : ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [41]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
      key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [42]:
# See what you just created
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [43]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]


In [44]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     1.     0.189  0.395
  0.782 -0.346  0.     1.   ]


In [45]:
# 6.) Build the Model

In [47]:
model = tf.keras.Sequential([
                             preprocessing_layer,
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(1)
])

model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [48]:
# 7.) Train, Evaluate and Predict

In [49]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [50]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 0s 2ms/step - loss: 0.5006 - accuracy: 0.7448
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4228 - accuracy: 0.8198
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3980 - accuracy: 0.8293
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3959 - accuracy: 0.8341
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3908 - accuracy: 0.8405
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3750 - accuracy: 0.8325
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3733 - accuracy: 0.8293
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3580 - accuracy: 0.8389
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3581 - accuracy: 0.8421
Epoch 10/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3505 - accuracy: 0.8501

In [51]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 2ms/step - loss: 0.4412 - accuracy: 0.8333


Test Loss 0.44118061661720276, Test Accuracy 0.8333333134651184


In [52]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 9.20%  | Actual outcome:  SURVIVED
Predicted survival: 72.21%  | Actual outcome:  SURVIVED
Predicted survival: 25.76%  | Actual outcome:  DIED
Predicted survival: 47.87%  | Actual outcome:  SURVIVED
Predicted survival: 11.68%  | Actual outcome:  DIED
